In [5]:
import torch
import torch.nn as nn
import rkgb
from rkgb.core.partitioned import PartitionerSequence, PartitionerBottomToTop
import rockmate
from rockmate import Rockmate
from rockmate.solvers import HILP, RK_rotor
from rockmate.solvers.main import add_sched

import sys
sys.path.append('/home/ygusak/rockmate-private/')
from models import get_nn_Transformer


/beegfs/ygusak/anaconda3/envs/py310_neuralop/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
device='cpu'

batchsize = 8

# model = torch.nn.Transformer(num_encoder_layers=1,num_decoder_layers=1)
# sample = [
#     torch.rand((200, batchsize, 512),device=device),
#     torch.rand((200, batchsize, 512),device=device),
# ]

model, sample = get_nn_Transformer(
    num_encoder_layers=1,
    num_decoder_layers=1,
    batchsize=8,
    device = device)
model.to(device)




cuda


/beegfs/ygusak/anaconda3/envs/py310_neuralop/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Transformer(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): TransformerDecoder(
    (layers): ModuleList(
      (0): TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=

In [10]:
# Partitioner to get a sequence of blocks
# sub_partitioner = PartitionerBottomToTop()
# sub_partitioner.config.max_estimate_for_main_graph=10**7
# sub_partitioner.config.max_estimate_per_sub_graph=10**6

# partitioner = PartitionerSequence(sub_partitioner=sub_partitioner)

In [11]:
# Build graphs based on the partitioner
rkgb_res = rkgb.rkgb.Result(
                    model,
                    model_args=sample,
                    # model_kwargs=model_kwargs,
                    # verbose=verbose,
                    # wanted_graphs={"FB"},
                    # partitioners=[partitioner],
                    inspection_device=torch.device("cuda"),
                    # print_time_in_each_stage=True
                )

/beegfs/ygusak/anaconda3/envs/py310_neuralop/lib/python3.10/site-packages/torch/cuda/memory.py:330: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [5]:
# for hcn in rkgb_res.hierarchical_cluster.partitionings[0].list_HCNs:
#     print(hcn.sub_cluster)
#     try:
#         print(hcn.sub_cluster.list_schedules)
#     except:
#         print('None cluster!!!!!!!\n')

In [6]:
solver = HILP(ilp_solver="PULP_CBC_CMD")
solver.config.offload = False
solver.config.solve_only_top_level = False
solver.config.nb_total_nodes_top_level = 0
rk_solver = RK_rotor()
list_solvers = [solver, rk_solver]
partitioners = [rkgb.partitioned.PartitionerSequence(
        sub_partitioner=rkgb.partitioned.Partitioner())]

budget = 5*10**8
rkmod = Rockmate(
        model,
        sample,
        budget=budget,
        list_solvers=list_solvers,
        rkgb_res=None,
        solve_sched=False,
        # verbose=False,
        # ilp_solver="PULP_CBC_CMD",
        # ilp_time_limit=1 * 60 // 360,
        # ilp_time_limit_top=10 * 60,
        # model_kwargs=None,
        partitioners=partitioners,
        # max_size_S_graph_for_no_partitioning=40,
        # cpu_optim = torch.optim.Adam,
        # gpu_optim = torch.optim.Adam,
        # optim_kwargs = {},
        # minor_param_size = 10*1024**2,
    )

Using PULP_CBC_CMD to solve ILP
Stage Raw took 00:00:03
Stage Forward took 00:00:00
Stage Simplifications took 00:00:00
Stage Partitioning took 00:00:00


/beegfs/ygusak/anaconda3/envs/py310_neuralop/lib/python3.10/site-packages/torch/cuda/memory.py:330: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
/beegfs/ygusak/anaconda3/envs/py310_neuralop/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Stage Backward took 00:00:00
Stage Hierarchical took 00:00:00


In [7]:
for hcn in rkmod.rkgb_res.hierarchical_cluster.partitionings[0].list_HCNs:
    print(hcn.sub_cluster)
    try:
        print(hcn.sub_cluster.list_schedules)
    except:
        print('None cluster!!!!!!!\n')

BottomHCluster(__3_addmm)
[]
BottomHCluster(__9_clone)
[]
H_Cluster_2_Ano_id_2
[]
BottomHCluster(__41_clone_1)
[]
BottomHCluster(__44_addmm_1)
[]
BottomHCluster(__47_getitem_2)
[]
BottomHCluster(__48_add)
[]
BottomHCluster(__50_getitem_4)
[]
H_Cluster_3_Ano_id_3
[]
BottomHCluster(__66_getitem_11)
[]
BottomHCluster(__68_getitem_14)
[]
H_Cluster_4_Ano_id_4
[]
BottomHCluster(__177_getitem_32)
[]
H_Cluster_5_Ano_id_3
None
BottomHCluster(__193_getitem_39)
[]
BottomHCluster(__195_getitem_42)
[]
None
None cluster!!!!!!!

BottomHCluster(__195_getitem_42)
[]
BottomHCluster(__193_getitem_39)
[]
H_Cluster_5_Ano_id_3
None
BottomHCluster(__177_getitem_32)
[]
H_Cluster_4_Ano_id_4
[]
BottomHCluster(__68_getitem_14)
[]
BottomHCluster(__66_getitem_11)
[]
H_Cluster_3_Ano_id_3
[]
BottomHCluster(__50_getitem_4)
[]
BottomHCluster(__48_add)
[]
BottomHCluster(__47_getitem_2)
[]
BottomHCluster(__44_addmm_1)
[]
BottomHCluster(__41_clone_1)
[]
H_Cluster_2_Ano_id_2
[]
BottomHCluster(__9_clone)
[]
BottomHCluster(

In [8]:
rkmod.preprocess()

In [9]:
for hcn in rkmod.rkgb_res.hierarchical_cluster.partitionings[0].list_HCNs:
    print(hcn.sub_cluster)
    if hcn.sub_cluster is None:
        print('None cluster!!!!!!!\n')
    elif hcn.sub_cluster.list_schedules is None:
        print('None schedule!!!!!!!\n')
    else:
        print(hcn.sub_cluster.list_schedules, '\n')


BottomHCluster(__3_addmm)
[Op_sched takes 0.71 ms with 18.75048828125 MiB peak mem, Op_sched takes 1.12 ms with 18.75048828125 MiB peak mem] 

BottomHCluster(__9_clone)
[Op_sched takes 0.40 ms with 0.0 MiB peak mem, Op_sched takes 0.68 ms with 0.0 MiB peak mem] 

H_Cluster_2_Ano_id_2
[Op_sched takes 2.01 ms with 50.87890625 MiB peak mem, Op_sched takes 2.77 ms with 50.87890625 MiB peak mem] 

BottomHCluster(__41_clone_1)
[Op_sched takes 0.20 ms with 0.0 MiB peak mem, Op_sched takes 0.29 ms with 0.0 MiB peak mem] 

BottomHCluster(__44_addmm_1)
[Op_sched takes 0.44 ms with 6.25048828125 MiB peak mem, Op_sched takes 0.63 ms with 6.25048828125 MiB peak mem] 

BottomHCluster(__47_getitem_2)
[Op_sched takes 0.16 ms with 1.5625 MiB peak mem, Op_sched takes 0.22 ms with 1.5625 MiB peak mem] 

BottomHCluster(__48_add)
[Op_sched takes 0.15 ms with 0.0 MiB peak mem, Op_sched takes 0.20 ms with 0.0 MiB peak mem] 

BottomHCluster(__50_getitem_4)
[Op_sched takes 0.22 ms with 0.025390625 MiB peak mem

In [10]:
cluster = rkmod.rkgb_res.hierarchical_cluster
param_mem = sum(pnode.mem for pnode in cluster.parameter_nodes)
param_grad_mem = sum(pnode.mem for pnode in cluster.parameter_nodes if pnode.info.requires_grad)
act_budget = budget - param_mem - (1+rkmod.optimize_metrics["optimizer_states_size"]) * param_grad_mem

rkmod.solve_sched(act_budget)
rkmod.get_compiled_fct()

solving H_Cluster_2_Ano_id_2
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
CHAIN SOLUTION!!!!!!!!!!!!!
CHAIN SOLUTION!!!!!!!!!!!!!
CHAIN SOLUTION!!!!!!!!!!!!!
solving H_Cluster_3_Ano_id_3
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
CHAIN SOLUTION!!!!!!!!!!!!!
CHAIN SOLUTION!!!!!!!!!!!!!
CHAIN SOLUTION!!!!!!

In [10]:
# rkmod.preprocess()

for hcn in rkmod.rkgb_res.hierarchical_cluster.partitionings[0].list_HCNs:
    print(hcn.sub_cluster)

    if hcn.sub_cluster is None: continue
    if not hcn.is_fwd: continue
    if not hcn.sub_cluster.name.startswith('H'): continue
    
    solver = HILP(ilp_solver="PULP_CBC_CMD")
    solver.config.optimize_metrics = {"minor_param_size": 10*1024**2}

    list_sched = solver(hcn.sub_cluster)
    if hcn.sub_cluster.list_schedules is None:
        hcn.sub_cluster.list_schedules = []
    for sched in list_sched:
        add_sched(hcn.sub_cluster, sched)


BottomHCluster(__3_addmm)
BottomHCluster(__9_clone)
H_Cluster_2_Ano_id_2
Using PULP_CBC_CMD to solve ILP
solving H_Cluster_2_Ano_id_2
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
BottomHCluster(__41_clone_1)
BottomHCluster(__44_addmm_1)
BottomHCluster(__47_getitem_2)
BottomHCluster(__48_add)
BottomHCluster(__50_getitem_4)
H_Cluster_3_Ano_id_3
Using PULP_CBC_CMD to solve ILP
solving H_Cluster_3_Ano_id_3
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time limit 900
Nb comp: 403, T:13
time lim

In [38]:
for hcn in rkgb_res.hierarchical_cluster.partitionings[0].list_HCNs:
    print(hcn.sub_cluster)
    if hcn.sub_cluster is None:
        print('None cluster!!!!!!!\n')
    elif hcn.sub_cluster.list_schedules is None:
        print('None schedule!!!!!!!\n')
    else:
        print(hcn.sub_cluster.list_schedules, '\n')

BottomHCluster(__3_addmm)
[Op_sched takes 0.72 ms with 18.75048828125 MiB peak mem, Op_sched takes 1.13 ms with 18.75048828125 MiB peak mem] 

BottomHCluster(__9_clone)
[Op_sched takes 0.30 ms with 0.0 MiB peak mem, Op_sched takes 0.43 ms with 0.0 MiB peak mem] 

H_Cluster_2_Ano_id_2
[Op_sched takes 2.25 ms with 50.87890625 MiB peak mem, Op_sched takes 3.21 ms with 50.87890625 MiB peak mem] 

BottomHCluster(__41_clone_1)
[Op_sched takes 0.19 ms with 0.0 MiB peak mem, Op_sched takes 0.28 ms with 0.0 MiB peak mem] 

BottomHCluster(__44_addmm_1)
[Op_sched takes 0.44 ms with 6.25048828125 MiB peak mem, Op_sched takes 0.63 ms with 6.25048828125 MiB peak mem] 

BottomHCluster(__47_getitem_2)
[Op_sched takes 0.16 ms with 1.5625 MiB peak mem, Op_sched takes 0.23 ms with 1.5625 MiB peak mem] 

BottomHCluster(__48_add)
[Op_sched takes 0.15 ms with 0.0 MiB peak mem, Op_sched takes 0.20 ms with 0.0 MiB peak mem] 

BottomHCluster(__50_getitem_4)
[Op_sched takes 0.22 ms with 0.025390625 MiB peak mem

In [11]:
# rkmod.solve_sched(list_solvers=[RK_rotor()], recursive=False)

solver = RK_rotor()
solver.solve(rkmod.rkgb_res.hierarchical_cluster, [5*10**7])

[]

In [12]:
rkmod.op_sched, rkmod.list_solutions

(None, [])

In [13]:
hcluster = rkmod.rkgb_res.hierarchical_cluster
hcluster.name

'H_Cluster_1_Ano_id_1'

In [21]:
rkmod.get_compiled_fct()


AttributeError: 'NoneType' object has no attribute 'simulate_update'